# Задание по дисциплине «Информационная безопасность и кодирование»
## Лабораторная работа №3 «Криптоанализ шифра простой замены»

ФИО:   Cавенко Екатерина Игоревна

Группа: БИВТ-21-4

### Импорт библиотек

In [ ]:
import pandas as pd
from itertools import product

### Зашифрованное сообщение

In [159]:
coded_message = "Ъыщбпьь шпцжтй ъщшйэж, щьшкшщмум пнщ."

### Функция для форматирования чисел

In [160]:
def format_freq(freq):
    return f"{freq:.4f}".replace('.', ',')

### Частотный анализ шифрованного текста

In [ ]:
coded_message_upper = coded_message.upper()
freq_dict = {}
for char in coded_message_upper:
    if char in [' ', ',', '.']:
        continue
    freq_dict[char] = freq_dict.get(char, 0) + 1

total_chars = sum(freq_dict.values())
data = []
for char, count in freq_dict.items():
    freq = count / total_chars
    data.append([char, freq]) 

df_frequencies = pd.DataFrame(data, columns=["Буква", "Частота"])
df_frequencies = df_frequencies.sort_values(by="Частота", ascending=False).reset_index(drop=True)
df_frequencies.index = range(1, len(df_frequencies) + 1)


df_for_csv = df_frequencies.copy()
df_for_csv["Частота"] = df_for_csv["Частота"].apply(format_freq)
df_for_csv.to_csv('char_frequencies.csv', sep=';', index=False, encoding='utf-8-sig')
print("Сохранен файл char_frequencies.csv с частотами символов")

print("\nТаблица частот:")
display(df_frequencies)

Сохранен файл char_frequencies.csv с частотами символов

Таблица частот:


,Буква,Частота
1,Щ,0.161290
2,Ш,0.129032
3,П,0.096774
4,Ь,0.096774
5,Ъ,0.064516
6,Ж,0.064516
7,Й,0.064516
8,М,0.064516
9,Ы,0.032258
10,Б,0.032258


### Ранжирование символов

In [162]:
# Функция для назначения рангов с учетом одинаковых частот
def assign_ranks(df):
    df = df.copy()
    df['temp_freq'] = df['Частота'].round(6)
    # Присваиваем ранги (метод 'dense' для одинаковых рангов при совпадении)
    df['Ранг'] = df['temp_freq'].rank(method='dense', ascending=False).astype(int)
    df = df.sort_values(by=['Ранг', 'Буква'])
    return df.drop(columns='temp_freq')

# Создаем датафрейм с рангами (все символы)
ranked_df = assign_ranks(df_frequencies)

# Фильтруем символы с частотой > 0.04
high_freq_df = ranked_df[ranked_df["Частота"] > 0.04]
high_freq_df.index = range(1, len(high_freq_df)+1)

# Сохраняем ранжированные данные
high_freq_df[['Буква','Ранг']].to_csv('ranked_frequencies.csv', sep=';', index=False, encoding='utf-8-sig')

print("\nРанжирование символов с частотой > 0.04:")
display(high_freq_df[['Буква','Ранг']])
print("Сохранен файл ranked_frequencies.csv c ранжированными частотами (> 0.04)")


Ранжирование символов с частотой > 0.04:


,Буква,Ранг
1,Щ,1
2,Ш,2
3,П,3
4,Ь,3
5,Ж,4
6,Й,4
7,М,4
8,Ъ,4


Сохранен файл ranked_frequencies.csv c ранжированными частотами (> 0.04)


### Замена символов согласно таблице частоты появления символов в среднестатистическом тексте для русского языка

In [ ]:
# Возможные замены по рангам
possible_replacements = {
    1: ['О'],
    2: ['Е', 'А'],
    3: ['А', 'И'],
    4: ['Н', 'Т']
}

# Функция для замены символов
def apply_replacement(text, replacement_dict):
    result = []
    for char in text:
        upper_char = char.upper()
        if upper_char in replacement_dict:
            new_char = replacement_dict[upper_char]
            result.append(new_char.lower() if char.islower() else new_char)
        else:
            result.append(char)
    return ''.join(result)

# Генерация всех возможных комбинаций замен
def generate_all_replacements(cipher_ranking, possible_replacements):
    # Группируем буквы по рангам
    ranked_chars = cipher_ranking.groupby('Ранг')['Буква'].apply(list).to_dict()
    
    # Для каждого ранга получаем возможные замены
    replacement_options = []
    for rank in sorted(ranked_chars.keys()):
        chars = ranked_chars[rank]
        replacements = possible_replacements.get(rank, ['?'])
        replacement_options.append((chars, replacements))
    
    # Генерируем все возможные комбинации
    all_combinations = []
    for chars, replacements in replacement_options:
        combos = list(product(chars, replacements))
        all_combinations.append(combos)
    
    return list(product(*all_combinations))

# Генерируем все комбинации
all_replacements = generate_all_replacements(high_freq_df, possible_replacements)

# Создаем список для хранения всех DataFrame
results_dfs = []

# Ограничим количество выводимых вариантов для наглядности
max_combinations_to_show = 5

for i, replacement_combo in enumerate(all_replacements[:max_combinations_to_show], 1):
    # Создаем словарь замен
    replacement_dict = {}
    replacement_data = []
    
    for char_replace_pairs in replacement_combo:
        char, replace = char_replace_pairs
        replacement_dict[char] = replace
        replacement_data.append([char, replace])
    
    # Создаем DataFrame для текущего варианта
    df = pd.DataFrame(replacement_data, columns=['Буква', 'Замена'])
    
    # Применяем замену к строке
    decoded = apply_replacement(coded_message, replacement_dict)
    
    # Добавляем столбец с результатом
    df['Строка'] = decoded
    
    # Добавляем в список результатов
    results_dfs.append((i, df))
    
    print(f"\nВариант {i}:")
    display(df)
    print(f"Полный результат: {decoded}\n")


# Сохраняем все варианты в Excel файл
with pd.ExcelWriter('change_results.xlsx') as writer:
    for i, df in results_dfs:
        df.to_excel(writer, sheet_name=f'Вариант {i}', index=False)

print("\nВсе варианты сохранены в файл 'change_results.xlsx'")


Вариант 1:


,Буква,Замена,Строка
0,Щ,О,"Ъыобаьь еацнтй ъоейэн, оьекеомум ано."
1,Ш,Е,"Ъыобаьь еацнтй ъоейэн, оьекеомум ано."
2,П,А,"Ъыобаьь еацнтй ъоейэн, оьекеомум ано."
3,Ж,Н,"Ъыобаьь еацнтй ъоейэн, оьекеомум ано."


Полный результат: Ъыобаьь еацнтй ъоейэн, оьекеомум ано.


Вариант 2:


,Буква,Замена,Строка
0,Щ,О,"Ъыобаьь еацттй ъоейэт, оьекеомум ано."
1,Ш,Е,"Ъыобаьь еацттй ъоейэт, оьекеомум ано."
2,П,А,"Ъыобаьь еацттй ъоейэт, оьекеомум ано."
3,Ж,Т,"Ъыобаьь еацттй ъоейэт, оьекеомум ано."


Полный результат: Ъыобаьь еацттй ъоейэт, оьекеомум ано.


Вариант 3:


...

Полный результат: Ъыобаьь еацжтн ъоенэж, оьекеомум ано.


Вариант 4:


,Буква,Замена,Строка
0,Щ,О,"Ъыобаьь еацжтт ъоетэж, оьекеомум ано."
1,Ш,Е,"Ъыобаьь еацжтт ъоетэж, оьекеомум ано."
2,П,А,"Ъыобаьь еацжтт ъоетэж, оьекеомум ано."
3,Й,Т,"Ъыобаьь еацжтт ъоетэж, оьекеомум ано."


Полный результат: Ъыобаьь еацжтт ъоетэж, оьекеомум ано.


Вариант 5:


,Буква,Замена,Строка
0,Щ,О,"Ъыобаьь еацжтй ъоейэж, оьекеонун ано."
1,Ш,Е,"Ъыобаьь еацжтй ъоейэж, оьекеонун ано."
2,П,А,"Ъыобаьь еацжтй ъоейэж, оьекеонун ано."
3,М,Н,"Ъыобаьь еацжтй ъоейэж, оьекеонун ано."


Полный результат: Ъыобаьь еацжтй ъоейэж, оьекеонун ано.


Все варианты сохранены в файл 'change_results.xlsx'


### Расшифровка по шифру Цезаря с произвольным смещением

In [164]:
# Подготовка списка слов (удаление знаков препинания)
coded_message_list = []
for word in coded_message.split(' '):
    word = word.strip('., ')
    if word:
        coded_message_list.append(word)

# Находим самое короткое слово в нижнем регистре
shortest_word = min(coded_message_list, key=len).lower()
message = shortest_word

# Русский алфавит
russian_alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

def caesar_decrypt(text, shift):
    decrypted = []
    for char in text:
        is_lower = char.islower()
        char = char.lower()  
        if char in russian_alphabet:
            # Находим индекс буквы в алфавите
            index = russian_alphabet.index(char)
            # Вычисляем новый индекс с учетом смещения
            new_index = (index - shift) % 33
            if is_lower:
                decrypted.append(russian_alphabet[new_index])
            else:
                decrypted.append(russian_alphabet[new_index].upper())
        else:
            decrypted.append(char)  # оставляем символ как есть, если не буква
    return ''.join(decrypted)

# Создаем список для хранения результатов
results = []

# Перебираем все возможные смещения от 1 до 33
for k in range(1, 34):
    decrypted_word = caesar_decrypt(message, k)
    results.append({'k': k, 'decrypted_message': decrypted_word})

# Создаем DataFrame и сохраняем в Excel
df = pd.DataFrame(results)
df.to_excel('decrypted_results.xlsx', index=False, engine='openpyxl')

print("Результаты сохранены в decrypted_results.xlsx")

Результаты сохранены в decrypted_results.xlsx


### Полная дешифровка по шифру Цезаря со смещением 11

In [165]:
k = 11
decrypted_word = caesar_decrypt(coded_message, 11)

print(f'Зашифрованное сообщение: {coded_message}')
print(f'Дешифрованное сообщение: {decrypted_word}')

Зашифрованное сообщение: Ъыщбпьь шпцжтй ъщшйэж, щьшкшщмум пнщ.
Дешифрованное сообщение: Процесс нельзя понять, оснановив его.
